<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_priprema_provera3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 12

Vežba obuhvata zadatke:
*   Zadatak 1 - Redukcija niza

## Preduslovi
Pre nego započnete implementaciju zadataka, **obavezno** izvršite kod iz ćelije ispod. Kod će:
- skinuti kompresovanu biblioteku `libwb` koja se koristi u zadacima i raspakovati je na putanju /usr/local/libwb.
- skinuti set podataka za testiranje rešenja zadatka i raspakovati ga na putanju /home/cuda

In [ ]:
%%bash

######################### PREUZMI I PODESI libwb ##############################
# preuzmi libwb.zip ako već nije preuzet
[[ -f libwb.zip ]] || wget -O libwb.zip "https://drive.google.com/u/0/uc?id=1-j31qXJJOBFKuNc4qzTjTU-Gji8HARh0&export=download"

# otpakuj
unzip -oqqd /usr/local libwb.zip


################### PREUZMI I PODESI set za testiranje ########################
# preuzmi dataset.zip ako vec nije preuzet
[[ -f dataset.zip ]] || wget -O dataset.zip "https://drive.google.com/u/0/uc?id=1I9lBg2p0zw4al7r9OJrKS_3Fp36Ho8pz&export=download"

# otpakuj u direktorijum u kojem ce se nalaziti izvrsna datoteka resenja
mkdir -p /home/cuda/
unzip -oqqd /home/cuda/ dataset.zip

Po završetku prethodne ćelije, možete pokrenuti narednu koja proverava da li `libwb` i `dataset` direktorijumi postoje na predviđenim putanjama. Ukoliko je sve u redu, obe naredbe će ispisati absolutne putanje do direktorijuma. U suprotnom će prijaviti da jedan ili oba direktorijuma nisu pronađena. U tom slučaju treba da se vratite na log prethodne ćelije i potražite gde je došlo do greške.

In [ ]:
%%bash

ls -d /usr/local/libwb/
ls -d /home/cuda/dataset

## [Opciono] Deinstalacija preduslova
Izvršavanjem ćelije ispod brišete instaliranu biblioteku `libwb` i set podataka. Izvršite ovu ćeliju samo ako želite da resetujete instalaciju kako biste reinstalirali preduslove.

In [ ]:
%%bash

# obrisi libwb biblioteku
[[ -d /usr/local/libwb ]] && rm -r /usr/local/libwb/ && echo "Obrisana libwb biblioteka."
[[ -f libwb.zip ]] && rm libwb.zip && echo "Obrisana libwb.zip arhiva."

# obrisi set podataka za testiranje
[[ -d /home/cuda/dataset ]] && rm -r /home/cuda/dataset && echo "Obrisan set podataka."
[[ -f dataset.zip ]] && rm dataset.zip && echo "Obrisana dataset.zip arhiva."

## Zadatak 1 - Redukcija niza

Implementirati CUDA kernel za redukciju ulaznog jednodimenzionalnog niza sumiranjem. Ulazni niz može biti proizvoljne dužine koja ne mora biti umnožak broja blokova niti. U tom slučaju, niti poslednjeg bloka mogu ostati neaktivne ukoliko ne postoji element niza koji treba da obrade ili se nedostajući element može posmatrati kao element vrednosti neutralne za zadati redukcioni operator (0 u ovom slučaju). Dužina ulaznog niza će biti sračunata na osnovu odabranog ulaznog niza i dostupna kao vrednost promenljive `numInputElements`. Svaki od blokova niti treba da sračuna redukcionu vrednost za svoj deo ulaznog niza po algoritmu koji treba implementirati u kernelu. Svaki blok po završetku kernela daje jednu izlaznu vrednost koja predstavlja parcijalnu redukciju dela niza koji je obrađivao. Sračunatu vrednost neka od niti bloka upisuje na odgovarajuću poziciju izlaznog niza koji je zadat kao parametar kernela i čija je dužina takođe unapred sračunata i dostupna kao sadržaj promenljive `numOutputElements`. Kao izlaznu vrednost, kernel vraća niz parcijalnih redukcija na osnovu kojih se računa konačna redukcija. Konačna redukcija niza se računa u kodu domaćina na osnovu parcijalnih redukcija sračunatih na uređaju (videti postavku zadatka na liniji 66). Po završetku implementacije obeleženih delova postavke zadatka, na početku datoteke u komentaru rečima opisati korake implementiranog kernela za redukciju.

Dopuniti zadatu postavku zadatka implementacijom nedostajućih delova
koda. Celokupno rešenje treba da se nađe u ćeliji ispod. Implementirati:

-   alokaciju memorije na uređaju,

-   kopiranje podataka iz memorije domaćina u memoriju uređaja,

-   inicijalizaciju dimenzija mreže i bloka,

-   poziv kernela,

-   kopiranje podataka sa memorije uređaja u memoriju domaćina i

-   dealokaciju memorije uređaja,

-   konvolucioni kernel

In [ ]:
%%writefile /home/cuda/array_reduction.cu

#include <wb.h>

#define BLOCK_SIZE 512 //@@ You can change this

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      return -1;                                                          \
    }                                                                     \
  } while (0)


__global__ void total(float *input, float *output, int len) {
  //@@ TODO implementirati kernel
}


int main(int argc, char **argv) {
  int ii;
  wbArg_t args;
  float *hostInput;      // The input 1D list
  float *hostOutput;     // The output list
  float *deviceInput;
  float *deviceOutput;
  int numInputElements;  // number of elements in the input list
  int numOutputElements; // number of elements in the output list

  args = wbArg_read(argc, argv);

  wbTime_start(Generic, "Importing data and creating memory on host");
  hostInput =
      (float *)wbImport(wbArg_getInputFile(args, 0), &numInputElements);

  numOutputElements = numInputElements / (BLOCK_SIZE << 1);
  if (numInputElements % (BLOCK_SIZE << 1)) {
    numOutputElements++;
  }
  hostOutput = (float *)malloc(numOutputElements * sizeof(float));

  wbTime_stop(Generic, "Importing data and creating memory on host");

  wbLog(TRACE, "The number of input elements in the input is ",
        numInputElements);
  wbLog(TRACE, "The number of output elements in the input is ",
        numOutputElements);

  wbTime_start(GPU, "Allocating GPU memory.");
  //@@ Allocate GPU memory here
  wbTime_stop(GPU, "Allocating GPU memory.");

  wbTime_start(GPU, "Copying input memory to the GPU.");
  //@@ Copy memory to the GPU here
  wbTime_stop(GPU, "Copying input memory to the GPU.");
  //@@ Initialize the grid and block dimensions here
  wbTime_start(Compute, "Performing CUDA computation");
  //@@ Launch the GPU Kernel here
  cudaDeviceSynchronize();
  wbTime_stop(Compute, "Performing CUDA computation");

  wbTime_start(Copy, "Copying output memory to the CPU");
  //@@ Copy the GPU memory back to the CPU here
  wbTime_stop(Copy, "Copying output memory to the CPU");

  for (ii = 1; ii < numOutputElements; ii++) {
    hostOutput[0] += hostOutput[ii];
  }

  wbTime_start(GPU, "Freeing GPU Memory");
  //@@ Free the GPU memory here
  wbTime_stop(GPU, "Freeing GPU Memory");

  wbSolution(args, hostOutput, 1);

  free(hostInput);
  free(hostOutput);

  return 0;
}


### Kompajliranje izvornog koda
Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/cuda
nvcc array_reduction.cu -o array_reduction \
  -I /usr/local/libwb/ \
  -L /usr/local/libwb/lib \
  -lwb -lcuda

### [Opciono] Provera da li postoji izvrsna datoteka

In [ ]:
![[ -f /home/cuda/array_reduction ]] && echo "Postoji." || echo "Ne postoji." 

### Pokretanje rešenja nad jednim primerom

In [ ]:
%%bash

cd /home/cuda
LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./array_reduction \
      -i dataset/0/input.raw \
      -o output.raw \
      -e dataset/0/output.raw \
      -t vector

### Pokretanje rešenja nad svim primerima

Nakon uspešnog testiranja rešenja na jednom primeru, možete pokrenuti i testiranje rešenja na svim primerima iz seta podataka.

In [ ]:
%%bash

cd /home/cuda

n=$(ls -dl dataset/* | wc -l)

for (( i=0; i<$n; i=i+1 ))
do
  echo "Testiram primer $i."
  echo "-------------------"
  LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./array_reduction \
      -i dataset/0/input.raw \
      -o output.raw \
      -e dataset/0/output.raw \
      -t vector
  echo ""
done